## Càrrega de dades

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_selection import SelectKBest, f_regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Ajustos visuals
sns.set_theme()
plt.rcParams['figure.figsize'] = (8, 5)

# Carreguem el CSV 
df = pd.read_csv(r"Path_to_your_file.csv")

print('Dimensions del datset:', df.shape)
#df.head(3)


In [ ]:
# Separació alumnat / professorat
role_col = "Quin és el teu rol a la teva institució educativa?"
student_df = df[df[role_col] == 'Estudiant'].copy()
teacher_df = df[df[role_col] != 'Estudiant'].copy()

print(f"Nombre d'estudiants: {student_df.shape[0]}")
print(f"Nombre de docents: {teacher_df.shape[0]}")

# Conversió de columnes Likert (1–5) a numèric
likert_cols = [c for c in df.columns if 'escala' in c or 'En quina mesura' in c or 'valoraries' in c]
for col in likert_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    student_df[col] = pd.to_numeric(student_df[col], errors='coerce')
    teacher_df[col] = pd.to_numeric(teacher_df[col], errors='coerce')


## 1. Estadística descriptiva per rol

### 1.1 Alumnat

In [ ]:
# ------------------------------------------------------------------
# 1) Nivell educatiu i antiguitat al centre
# ------------------------------------------------------------------
LEVEL_COL  = "Quin nivell d'estudis estàs cursant en aquest moment o en quin nivell dels següents ets docent?"
YEARS_COL  = "Quants anys portes a la teva institució educativa?"

level_counts = student_df[LEVEL_COL].value_counts().sort_index()
years_counts = student_df[YEARS_COL].value_counts().sort_index()

print("\n--- NIVELL EDUCATIU ---")
print(level_counts)
print("\n--- ANTIGUITAT ---")
print(years_counts)

N_student_df = student_df.shape[0]
# Percentatges simples per comprovar les frases de l'informe
pct_leq2 = years_counts.loc[["Menys d'1 any", "1-2 anys"]].sum() / N_student_df * 100
pct_3_4  = years_counts.get("3-4 anys", 0) / N_student_df * 100
print('\n Stasts de l\'antiguitat:')
print(f"Alumnes ≤2 anys: {pct_leq2:.1f}% ; 3–4 anys: {pct_3_4:.1f}%")

In [ ]:
# ------------------------------------------------------------------
# 2) Coneixement i freqüència d'ús d'IA
# ------------------------------------------------------------------
KNOW_COL = "En una escala de l’1 al 5, com valoraries el teu coneixement i comprensió de les eines d’IA (per exemple, ChatGPT) per a finalitats acadèmiques?"
FREQ_COL = "En una escala de l’1 al 5, amb quina freqüència utilitzes eines d’IA (per exemple, ChatGPT) per a tasques acadèmiques?"

know_mean = student_df[KNOW_COL].mean()
freq_mean = student_df[FREQ_COL].mean()

pct_know_high = (student_df[KNOW_COL] >= 4).mean() * 100
pct_freq_high = (student_df[FREQ_COL] >= 4).mean() * 100

print("\n--- CONEIXEMENT I FREQÜÈNCIA ---")
print(f"Mitjana coneixement IA: {know_mean:.2f}")
print(f"% coneixement ≥4: {pct_know_high:.1f}%")
print(f"Mitjana freqüència ús IA: {freq_mean:.2f}")
print(f"% ús ≥4: {pct_freq_high:.1f}%")

In [ ]:
# Coneixement de la IA (1-5)
col_coneixement = "En una escala de l’1 al 5, com valoraries el teu coneixement i comprensió de les eines d’IA (per exemple, ChatGPT) per a finalitats acadèmiques?"
sns.histplot(student_df[col_coneixement].dropna(), bins=5, kde=False, color='skyblue')
plt.title('Coneixement de les eines d\'IA (Estudiants)')
plt.xlabel('Escala de coneixement')
plt.ylabel('Nombre de respostes')
plt.grid(True)
plt.show()

student_df[col_coneixement].describe()

# Freqüència d'ús (1-5)
col_frequencia = "En una escala de l’1 al 5, amb quina freqüència utilitzes eines d’IA (per exemple, ChatGPT) per a tasques acadèmiques?"
sns.boxplot(x=student_df[col_frequencia], color='lightgreen')
plt.title('Freqüència d\'ús de la IA (Estudiants)')
plt.xlabel('Escala de freqüència')
plt.grid(True)
plt.show()

student_df[col_frequencia].describe()

In [ ]:
# ------------------------------------------------------------------
# 3) Eines d'IA utilitzades (multi‐resposta separada per “;”)
# ------------------------------------------------------------------
TOOLS_COL = "Quines de les següents eines d’IA has utilitzat per a tasques acadèmiques? "
tools_series = student_df[TOOLS_COL].dropna().str.split(";").explode().str.strip()
tool_counts  = tools_series.value_counts(normalize=True).mul(100).round(1)

# Agrupa les opcions amb percentatge < 1% sota 'Altres'
main_tools = tool_counts[tool_counts >= 1]
extra_pct = tool_counts[tool_counts < 1].sum()
if extra_pct > 0:
    main_tools['Altres (<1%)'] = extra_pct

print("\n--- EINES D'IA MÉS USADES ---")
print(main_tools)

In [ ]:
# Eines d'IA utilitzades (multiresposta)
from collections import Counter

col_eines = "Quines de les següents eines d’IA has utilitzat per a tasques acadèmiques? "
eines_list = student_df[col_eines].dropna().str.split(';').sum()
eines_freq = Counter(eines_list)
eines_df = pd.DataFrame(eines_freq.items(), columns=['Eina', 'Usuaris']).sort_values(by='Usuaris', ascending=False)
eines_df["%"] = (eines_df["Usuaris"] / len(student_df)) * 100

eines_df_filtrat = eines_df[eines_df["%"] > 0.7]

ax = sns.barplot(data=eines_df_filtrat, y='Eina', x='Usuaris', palette='coolwarm')
plt.title('Eines d\'IA més utilitzades per estudiants')
plt.xlabel('Nombre d\'usuaris')
plt.ylabel('Eina')
plt.grid(True)

# Afegeix el percentatge a cada barra
for i, (usuari, pct) in enumerate(zip(eines_df_filtrat["Usuaris"], eines_df_filtrat["%"])):
    ax.text(usuari + 0.5, i, f"{pct:.1f}%", va='center', color='black')

plt.show()

In [ ]:
# ------------------------------------------------------------------
# 4) Contextos d'ús d'IA (multi‐resposta)
# ------------------------------------------------------------------
CTX_COL = "En quins dels següents contextos utilitzes eines d’IA? (Selecciona totes les opcions aplicables)"
ctx_series = student_df[CTX_COL].dropna().str.split(";").explode().str.strip()
ctx_counts = ctx_series.value_counts(normalize=True).mul(100).round(1)

print("\n--- CONTEXTOS D'ÚS ---")
print(ctx_counts.head(10))   # els més freqüents

n_contextes_per_student = student_df[CTX_COL].dropna().apply(lambda x: len(x.split(";"))).mean()
print(f"\nMitjana de contextos d'ús per estudiant: {n_contextes_per_student:.1f}")


In [ ]:
# Contextos d'ús (multiresposta)
col_contextos = "En quins dels següents contextos utilitzes eines d’IA? (Selecciona totes les opcions aplicables)"
context_list = student_df[col_contextos].dropna().str.split(';').sum()
context_freq = Counter(context_list)
context_df = pd.DataFrame(context_freq.items(), columns=['Context', 'Freqüència']).sort_values(by='Freqüència', ascending=False)

sns.barplot(data=context_df, y='Context', x='Freqüència', palette='viridis')
plt.title('Contextos més comuns d\'ús de la IA')
plt.xlabel('Nombre d\'usos')
plt.ylabel('Context')
plt.grid(True)
plt.show()

In [ ]:
# ------------------------------------------------------------------
# 5) Beneficis i dependència percebuts
# ------------------------------------------------------------------
BEN_COL      = "Creus que les eines d’IA (per exemple, ChatGPT) milloren la teva experiència general d’aprenentatge?"
REND_COL     = "En una escala de l’1 al 5, en quina mesura creus que les eines d’IA milloren el teu rendiment acadèmic?"
HELP_COL     = "En quina mesura creus que les eines d’IA t’ajuden a comprendre temes acadèmics complexos?"
DEP_COL      = "En quina mesura estàs d’acord amb la següent afirmació: 'Quan faig servir eines d’IA amb regularitat, tendeixo a dependre més d’elles per resoldre problemes i aprendre, en lloc de fer-ho amb les meves pròpies habilitats cognitives.'"
EXAM_COL     = "Si en un examen final et deixen escollir entre portar els teus apunts de l'assignatura o fer ús del ChatGPT, què triaries?"
NOIA_COL     = "Si no poguessis utilitzar eines d’IA durant un semestre, com creus que afectaria el teu rendiment acadèmic?"

# Benefici global
ben_counts = student_df[BEN_COL].value_counts(normalize=True).mul(100).round(1)
print("\n--- BENEFICI EXPERIÈNCIA GLOBAL ---")
print(ben_counts)

# Mitjana millora rendiment + % dependència alta
rend_mean = student_df[REND_COL].mean()
print(f"\nMitjana millora rendiment: {rend_mean:.2f}")

pct_help_high = (student_df[HELP_COL] >= 4).mean()*100
pct_dep_high  = (student_df[DEP_COL]  >= 4).mean()*100
print(f"% ajuda a comprendre (≥4): {pct_help_high:.1f}%")
print(f"% dependència percebut (≥4): {pct_dep_high:.1f}%")

# Examen: ChatGPT vs Apunts
exam_counts = student_df[EXAM_COL].value_counts(normalize=True).mul(100).round(1)
print("\n--- TRIAR CHATGPT EN EXAMEN ---")
print(exam_counts)

# Impacte sense IA un semestre (escala 1=empitjoraria molt, 5=milloraria)
noia_mean = student_df[NOIA_COL].mean()
pct_noia_high = (student_df[NOIA_COL] >= 4).mean()*100
pct_noia_low  = (student_df[NOIA_COL] <= 2).mean()*100
print(f"\nMitjana impacte sense IA: {noia_mean:.2f}")
print(f"% diu que baixaria molt (4–5): {pct_noia_high:.1f}%")
print(f"% diu que afectaria poc (1–2): {pct_noia_low:.1f}%")

In [ ]:
# Preferència: Apunts vs ChatGPT
col_pref = "Si en un examen final et deixen escollir entre portar els teus apunts de l'assignatura o fer ús del ChatGPT, què triaries?"
sns.countplot(y=student_df[col_pref], order=student_df[col_pref].value_counts().index)
plt.title('Preferència en Examen: Apunts vs ChatGPT')
plt.xlabel('Nombre d\'estudiants')
plt.ylabel('Opció triada')
plt.grid(True)
plt.show()

student_df[col_pref].value_counts(normalize=True)

sns.histplot(student_df[REND_COL].dropna(), bins=5, kde=False)
plt.title('Millora del rendiment acadèmic amb IA')


In [ ]:
# ------------------------------------------------------------------
# 6) Altres percepcions: creativitat, retenció, privacitat, interès i recomanació
# ------------------------------------------------------------------
CREA_COL  = "En quina mesura estàs d’acord amb la següent afirmació: 'Confiar massa en les eines d’IA (com ChatGPT) podria danyar la creativitat i les habilitats de pensament crític dels estudiants.'"
RET_COL   = "Després d’estudiar amb assistència d’IA, quant de temps retens normalment el material après en comparació amb mètodes d’estudi tradicionals?"
PRIV_COL  = "Com de preocupat estàs per la privadesa i la seguretat de les teves dades personals quan utilitzes eines d’IA per a tasques acadèmiques?"
INTER_COL = "T’interessaria fer servir una eina web que avaluï l’estat del teu procés d’aprenentatge segons el teu ús de la IA i el teu rendiment acadèmic?"
REC_COL   = "Quina probabilitat hi ha que recomanis l’ús d’eines d’IA per a finalitats acadèmiques als teus companys?"

pct_crea_concern = (student_df[CREA_COL] >= 4).mean()*100
pct_priv_high    = (student_df[PRIV_COL] >= 4).mean()*100
priv_low_mid     = (student_df[PRIV_COL] <= 3).mean()*100

ret_less  = (student_df[RET_COL] <= 2).mean()*100
ret_same  = ((student_df[RET_COL] == 3)).mean()*100
ret_more  = (student_df[RET_COL] >= 4).mean()*100

int_mean  = student_df[INTER_COL].mean()
rec_mean  = student_df[REC_COL].mean()

print("\n--- CREATIVITAT I DEPENDÈNCIA ---")
print(f"% preocupats per creativitat (≥4): {pct_crea_concern:.1f}%")
print("\n--- RETENCIÓ ---")
print(f"% retenció menys temps (≤2): {ret_less:.1f}%")
print(f"% retenció igual (3):        {ret_same:.1f}%")
print(f"% retenció més temps (≥4):   {ret_more:.1f}%")
print("\n--- PRIVACITAT ---")
print(f"% preocupació alta (≥4): {pct_priv_high:.1f}%")
print(f"% preocupació 1–3:       {priv_low_mid:.1f}%")
print("\n--- INTERÈS / RECOMANACIÓ ---")
print(f"Mitjana interès eina web: {int_mean:.2f}")
print(f"Mitjana probabilitat recomanar IA: {rec_mean:.2f}")


### 1.2 Professorat

In [ ]:
ROLE_COL   = "Quin és el teu rol a la teva institució educativa?"
PROF_DF    = df[df[ROLE_COL] == "Professor/Personal docent"].copy()
N_PROF     = len(PROF_DF)
print(f"Total professorat: {N_PROF}")

# ------------------------------------------------------------------
# 1) Perfil d’experiència i àrees
# ------------------------------------------------------------------
YEARS_COL  = "Quants anys portes a la docència?"
AREA_COL   = "Quina és la teva àrea d’estudi (o la matèria que ensenyes, si és aplicable)?"

years_counts = PROF_DF[YEARS_COL].value_counts().reindex(
    ["Menys d'1 any", "1-2 anys", "3-4 anys", "5+ anys"]
).fillna(0).astype(int)

print("\n--- ANTIGUITAT DOCENT ---")
print(years_counts)
print(f"% >5 anys: {years_counts['5+ anys']/N_PROF*100:.1f}%")
print(f"% 3–4 anys: {years_counts['3-4 anys']/N_PROF*100:.1f}%")
print(f"% ≤2 anys : {(years_counts["Menys d'1 any"]+years_counts['1-2 anys'])/N_PROF*100:.1f}%")

print("\n--- ÀREES DE CONEIXEMENT (Top 10) ---")
AREA_COL_CLEAN = PROF_DF[AREA_COL].str.strip().str.capitalize()
print(AREA_COL_CLEAN.value_counts().head(10))


# ------------------------------------------------------------------
# 2) Ús d’eines d’IA i tasques docents
# ------------------------------------------------------------------
# 2.1 Proporció que ha usat alguna IA
TOOLS_COL = "En quines de les següents tasques fas servir eines d’IA? (Selecciona totes les aplicables)"
has_ai = PROF_DF[TOOLS_COL].notna()
print(f"\n% professorat que utilitza IA en docència: {has_ai.mean()*100:.1f}%")

# 2.2 Comptem tasques (multi-resposta separada per “;”)
task_series = PROF_DF[TOOLS_COL].dropna().str.split(";").explode().str.strip()
task_counts = task_series.value_counts(normalize=True).mul(100).round(1)

print("\n--- TASQUES ON S’USA IA (Top) ---")
print(task_counts.head(10))

# ------------------------------------------------------------------
# 3) Formats d’avaluació i política d’ús d’IA
# ------------------------------------------------------------------
FORMAT_COL = "Quin format d’avaluació fas servir principalment en la teva assignatura?"
ALLOW_COL  = "L’element avaluatiu de més pes de la teva assignatura permet l’ús d’eines d’IA per completar-lo?"

fmt_counts   = PROF_DF[FORMAT_COL].value_counts(normalize=True).mul(100).round(1)
allow_counts = PROF_DF[ALLOW_COL].value_counts(normalize=True).mul(100).round(1)

print("\n--- FORMAT PRINCIPAL D’AVALUACIÓ ---")
print(fmt_counts)

print("\n--- POLÍTICA D’ÚS D’IA EN L’AVALUACIÓ ---")
print(allow_counts)
print(f"P ermeten IA (sense o amb límits): {(allow_counts.get('Sí, sense restriccions',0)+allow_counts.get('Sí, però amb limitacions específiques',0)):.1f}%")
print(f"Prohibeixen IA: {allow_counts.get('No, l’ús d’IA està prohibit',0):.1f}%")

# ------------------------------------------------------------------
# 4) Creença que l’alumne pot aprovar sense aprendre
# ------------------------------------------------------------------
CHEAT_COL = "Creus que l’ús de la IA permet a l’alumnat superar l’element avaluatiu de més pes de la teva assignatura sense haver adquirit realment els coneixements necessaris?"
cheat_mean = PROF_DF[CHEAT_COL].mean()
pct_cheat = (PROF_DF[CHEAT_COL] >= 4).mean()*100

print(f"\nMitjana «aprovar sense aprendre»: {cheat_mean:.2f} (escala 1-5)")
print(f"% que ho creu bastant (≥4): {pct_cheat:.1f}%")

# ------------------------------------------------------------------
# 5) Estratègies per validar l’autenticitat (multi-resposta)
# ------------------------------------------------------------------
STRAT_COL = "Quines estratègies utilitzes per validar l’autenticitat del treball de l’alumnat i calibrar el seu aprenentatge? (Selecciona totes les aplicables)"
strat_series = PROF_DF[STRAT_COL].dropna().str.split(";").explode().str.strip()
strat_counts = strat_series.value_counts(normalize=True).mul(100).round(1)

print("\n--- ESTRATÈGIES DE VALIDACIÓ (Top) ---")
print(strat_counts.head(10))

# ------------------------------------------------------------------
# 6) Percepcions sobre retenció i pensament crític
# ------------------------------------------------------------------
RET_COL   = "Segons la teva experiència, els estudiants que utilitzen freqüentment eines d’IA mostren una millor retenció de coneixement que aquells que no les utilitzen?"
CRIT_COL  = "Com percebeu l’impacte de la IA en la capacitat dels estudiants per pensar críticament i resoldre problemes de manera independent?"

ret_mean  = PROF_DF[RET_COL].mean()
pct_ret_pos = (PROF_DF[RET_COL] >= 4).mean()*100

crit_high = (PROF_DF[CRIT_COL] <= 2).mean()*100  # 1-2 = redueix
crit_low  = (PROF_DF[CRIT_COL] >= 4).mean()*100  # 4-5 = millora

print(f"\nMitjana retenció percebuda: {ret_mean:.2f} (1=gens, 5=molta)")
print(f"% docents que veuen millora clara (≥4): {pct_ret_pos:.1f}%")
print(f"% que veuen MENYS pensament crític (vals 1-2): {crit_high:.1f}%")
print(f"% que veuen més pensament crític (vals 4-5): {crit_low:.1f}%")

# ------------------------------------------------------------------
# 7) Necessitats de formació i recursos
# ------------------------------------------------------------------
KNOW_GAP_COL  = "Consideres que et falta coneixement sobre IA per aplicar-la a la teva docència?"
TOOLS_GAP_COL = "Consideres que et falten eines o recursos per gestionar l’ús de la IA a l’aula?"
SUPPORT_COL   = "Com valoraries el suport extern (eines, formació, assessorament) per integrar la IA en la teva docència?"
GUIDE_COL     = "Hi ha directrius clares en el teu centre educatiu sobre com usar les eines d'IA a les aules?"

know_gap_hi   = (PROF_DF[KNOW_GAP_COL] >= 4).mean()*100
tools_gap_hi  = (PROF_DF[TOOLS_GAP_COL] >= 4).mean()*100
support_mean  = PROF_DF[SUPPORT_COL].mean()
guidelines_pct = (PROF_DF[GUIDE_COL] == "Sí").mean()*100

print(f"\n% que admet mancança de coneixement (≥4): {know_gap_hi:.1f}%")
print(f"% que troba a faltar eines/recursos (≥4): {tools_gap_hi:.1f}%")
print(f"Mitjana valoració suport extern: {support_mean:.2f}/5")
print(f"% centres amb directrius clares: {guidelines_pct:.1f}%")

# ------------------------------------------------------------------
# 8) Suport institucional i recomanació
# ------------------------------------------------------------------
RES_COL     = "Com valoraries la disponibilitat de recursos i suport (per exemple, materials de formació, tutorials, serveis d’ajuda) per a l’ús d’eines d’IA a la teva institució educativa?"
REC_COL     = "Quina probabilitat hi ha que recomanis l’ús d’eines d’IA per a finalitats acadèmiques als teus companys?"

res_mean = PROF_DF[RES_COL].mean()
rec_mean = PROF_DF[REC_COL].mean()

print(f"\nMitjana recursos institucionals: {res_mean:.2f}/5")
print(f"Mitjana probabilitat recomanar IA: {rec_mean:.2f}/5")
